# Exercise: Performance Optimization

Optimize the following function.

In [1]:
function work!(A, B, v)
    val = zero(eltype(v))
    for i in 1:N
        val = mod(v[i],256)
        A[i,1:N] = B[i,1:N] * (sin(val) * sin(val) - cos(val) * cos(val))
    end
    return A
end

work! (generic function with 1 method)

The following data is **fixed** and **not supposed to be modified**!

In [2]:
# do not modify this cell!

using Random
Random.seed!(42)

N = 8000
B = rand(N,N)
v = rand(Int, N);

const result = work!(zeros(N,N), B, v);

# do not modify this cell!

You can compare against `A_result` to test your implementation(s):

In [3]:
using Test

@test work!(zeros(N,N), B, v) ≈ result

Test Passed

You can benchmark as follows:

In [4]:
using BenchmarkTools

@btime work!(A, $B, $v) setup=(A=zeros(N,N)); # or use @benchmark for more information

  2.372 s (134979 allocations: 979.23 MiB)


## Your Optimizations

Your optimized variants go here!

**Comments:**

* Improving the code doesn't necessarily give a noticable speedup immediately but it will enable further optimization.
* On my machine, I could reach a ~30x speedup.

<details>

<summary><b>Hints</b> (click to unfold)</summary>

* Try to look for type instabilities (with `@code_warntype`).
* Try to avoid unnecessary allocations (keyword: slicing).
* Try to optimize the memory access (keyword: column-major order).
* Bonus: Simplify the algebra by using a trigonometric identity.

</details>

### Fixing the type instability (accessing global `N`)

In [5]:
@code_warntype work!(zeros(N,N), B, v)

MethodInstance for work!(::Matrix{Float64}, ::Matrix{Float64}, ::Vector{Int64})
  from work!(A, B, v) @ Main In[1]:1
Arguments
  #self#::Core.Const(work!)
  A::Matrix{Float64}
  B::Matrix{Float64}
  v::Vector{Int64}
Locals
  @_5::Any
  val::Any
  i::Any
Body::Matrix{Float64}
1 ─       Core.NewvarNode(:(@_5))
│         Core.NewvarNode(:(val))
│   %3  = Main.size(A)::Tuple{Int64, Int64}
│   %4  = Main.size(B)::Tuple{Int64, Int64}
│   %5  = (%3 == %4)::Bool
└──       goto #3 if not %5
2 ─       goto #4
3 ─ %8  = Base.AssertionError("size(A) == size(B)")::Core.Const(AssertionError("size(A) == size(B)"))
└──       Base.throw(%8)
4 ┄ %10 = Main.eltype(v)::Core.Const(Int64)
│         (val = Main.zero(%10))
│   %12 = (1:Main.N)::Any
│         (@_5 = Base.iterate(%12))
│   %14 = (@_5 === nothing)::Bool
│   %15 = Base.not_int(%14)::Bool
└──       goto #7 if not %15
5 ┄ %17 = @_5::Any
│         (i = Core.getfield(%17, 1))
│   %19 = Core.getfield(%17, 2)::Any
│   %20 = Base.getindex(v, i)::Any
│  

Get `N` from the size of `A` (or `B`) or add another function argument.

In [6]:
function work1!(A, B, v)
    N = size(A,1) # or additional function argument
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        A[i,1:N] = B[i,1:N] * val
    end
    return A
end

@btime work1!(A, $B, $v) setup=(A=zeros(N,N))
@test work1!(zeros(N,N), B, v) ≈ result

  2.531 s (32000 allocations: 977.29 MiB)


Test Passed

### Analytic optimization (style points 😉)

Trigonometric identity

In [7]:
x = rand()
@test sin(x) * sin(x) - cos(x) * cos(x) ≈ -cos(2*x)

Test Passed

In [8]:
function work2!(A, B, v)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        A[i,1:N] = B[i,1:N] * val
    end
    return A
end

@btime work2!(A, $B, $v) setup=(A=zeros(N,N))
@test work2!(zeros(N,N), B, v) ≈ result

  2.262 s (32000 allocations: 977.29 MiB)


Test Passed

### Avoid allocations due to slicing (`B[i, 1:N]`)

In [9]:
function work3_vectorized!(A, B, v)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))
        @views A[i,1:N] = B[i,1:N] * val
    end
    return A
end

@btime work3_vectorized!(A, $B, $v) setup=(A=zeros(N,N))
@test work3_vectorized!(zeros(N,N), B, v) ≈ result

  2.006 s (16000 allocations: 488.65 MiB)


Test Passed

Same idea but explicit loop + `@inbounds`

In [10]:
function work3_loop!(A, B, v)
    N = size(A,1)
    val = zero(eltype(v))
    for i in 1:N
        val = -cos(2*mod(v[i],256))     
        for j in 1:N
            @inbounds A[i,j] = B[i,j] * val
        end
    end
    return A
end

@btime work3_loop!(A, $B, $v) setup=(A=zeros(N,N))
@test work3_loop!(zeros(N,N), B, v) ≈ result

  1.468 s (0 allocations: 0 bytes)


Test Passed

### Separating `val` computation

In [11]:
function work4_vectorized!(A, B, v)
    N = size(A,1)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for i in 1:N
        @views A[i,1:N] .= B[i,1:N] .* val[i]
    end
    return A
end

@btime work4_vectorized!(A, $B, $v) setup=(A=zeros(N,N))
@test work4_vectorized!(zeros(N,N), B, v) ≈ result

  1.509 s (2 allocations: 62.55 KiB)


Test Passed

Same idea but explicit loop + `@inbounds`

In [12]:
function work4_loop!(A, B, v)
    N = size(A,1)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for i in 1:N
        for j in 1:N
            @inbounds A[i,j] = B[i,j] * val[i]
        end
    end
    return A
end

@btime work4_loop!(A, $B, $v) setup=(A=zeros(N,N))
@test work4_loop!(zeros(N,N), B, v) ≈ result

  1.499 s (2 allocations: 62.55 KiB)


Test Passed

### Switch loop order (!!!)

In [13]:
function work5!(A, B, v)
    N = size(A,1)
    val = [-cos(2*mod(x,256)) for x in v]
    
    for j in 1:N
        for i in 1:N
            @inbounds A[i,j] = B[i,j] * val[i]
        end
    end
    return A
end

@btime work5!(A, $B, $v) setup=(A=zeros(N,N))
@test work5!(zeros(N,N), B, v) ≈ result

  72.985 ms (2 allocations: 62.55 KiB)


Test Passed

And here with broadcasting, for comparison:

In [19]:
function work6!(A, B, v)
    val = @. -cos(2*mod(v,256))
    @. A = B * val
    return A
end

@btime work6!(A, $B, $v) setup=(A=zeros(N,N))
@test work6!(zeros(N, N), B, v) ≈ result

  72.576 ms (2 allocations: 62.55 KiB)


Test Passed

## Bonus Question: Performance limit?

Look at your final optimized version of `work!`.

* In the limit of larger `A` and `B`, what is conceptually limiting the performance, the compute capability or memory transfer (i.e. reading and writing `A` and `B`)?

Let's try to quickly estimate the maximal memory bandwidth that a single-CPU core can achieve on the given computer:

In [14]:
using STREAMBenchmark
membw = memory_bandwidth(; nthreads=1).median / 1000 # memory bandwidth in GB /s

32.945699999999995

For references, a single CPU-core in [Noctua 2](https://pc2.uni-paderborn.de/systems-and-services/noctua-2) can achieve a **maximal memory bandwidth of ~45 GB/s**.

* Given the maximal memory bandwidth, can you give a performance bound estimate, i.e. the minimal runtime that we could possibly hope to achieve?
  * Hint: how many flops are performed per iteration and how many bytes are transferred?
* How far off is your implementation from achieving the limit (in percent)?

In [15]:
# membw = 45 # GB/s
flops = 1 # flops per iteration
traffic = 3*8 # bytes per iteration
I = flops / traffic # flops / byte

perf_bound = I*membw # GFLOPS
runtime_estimate = N^2 * 1e3 / (perf_bound * 1e9) # in ms

println("Performance bound: ", round(perf_bound, digits=2), " GFLOP/s")
println("Runtime estimate: ", round(runtime_estimate, digits=2), " ms")

Performance bound: 1.37 GFLOP/s
Runtime estimate: 46.62 ms


In [16]:
t_work5 = @belapsed work5!(A, $B, $v) setup=(A=zeros(N,N))
ratio = runtime_estimate / (t_work5 * 1e3)
println("My best version achieves ", round(ratio * 100, digits=2), "% of the limit.")

My best version achieves 64.26% of the limit.
